# Import libraries

In [ ]:
# Fix randomness and hide warnings
seed = 42

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['MPLCONFIGDIR'] = os.getcwd()+'/configs/'

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

import numpy as np
np.random.seed(seed)

import logging

import random
random.seed(seed)
import datetime

In [ ]:
!pip install tensorflow==2.14.0 -U -q
import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)
print(tf.__version__)

In [ ]:
# Import tensorflow
!pip install keras-cv keras-tuner -q -U

import keras_cv
import keras_tuner as kt

In [ ]:
# Import other libraries
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split, KFold
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, roc_curve, auc
import seaborn as sns

# Splitting

In [ ]:
processed_data = np.load('/kaggle/input/processed-data/processed_data.npz')
X = processed_data['X']
y = processed_data['y']
labels = {0:'healthy', 1:'unhealthy'}
y.shape

In [ ]:
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X,
    y,
    test_size=0.1,
    stratify=y,
    random_state=seed
)
# Further split the combined training and validation set into a training set and a validation set
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val,
    y_train_val,
    test_size = len(X_test), # Ensure validation set size matches test set size
    stratify=y_train_val,
    random_state=seed
)

# Print the shapes of the resulting datasets
print("Training_Validation Data Shape:", X_train_val.shape)
print("Training_Validation Label Shape:", y_train_val.shape)
print("Train Data Shape:", X_train.shape)
print("Train Label Shape:", y_train.shape)
print("Validation Data Shape:", X_val.shape)
print("Validation Label Shape:", y_val.shape)
print("Test Data Shape:", X_test.shape)
print("Test Label Shape:", y_test.shape)

In [ ]:
# Display the count of occurrences of target classes in the training-validation dataset
print('Counting occurrences of y_train classes:')
print(pd.DataFrame(np.argmax(y_train, axis=-1), columns=['class']).value_counts())

print('Counting occurrences of y_val classes:')
print(pd.DataFrame(np.argmax(y_val, axis=-1), columns=['class']).value_counts())

print('Counting occurrences of y_test classes:')
print(pd.DataFrame(np.argmax(y_test, axis=-1), columns=['class']).value_counts())


In [ ]:
# Compute alpha for the CategoricalFocalCrossentropy loss function
# It may be set by inverse class frequency by using compute_class_weight from sklearn.utils. 
class_weight=compute_class_weight('balanced', classes=np.unique(y_train), y=np.argmax(y_train, axis=-1))
class_weight_dict = dict(zip(np.unique(y_train.astype('int')), class_weight))
alpha = 1 / class_weight
print(class_weight, class_weight_dict, alpha)

In [ ]:
# Define batch size, number of epochs, learning rate, input shape, and output shape
AUTOTUNE = tf.data.AUTOTUNE
batch_size = 16
epochs = 200
es_patience = 10
rp_patience = 20
rp_min_lr = 1e-5
rp_factor = 0.1


input_shape = X_train.shape[1:]
output_shape = 2

# Define two callback functions for early stopping and learning rate reduction
callbacks=[
    tfk.callbacks.EarlyStopping(monitor='val_accuracy', patience=es_patience, restore_best_weights=True, mode='max'),
    tfk.callbacks.ReduceLROnPlateau(monitor="val_accuracy", factor=rp_factor, patience=rp_patience, min_lr=rp_min_lr, mode='max')
]

# Augmentation
Perform Advanced Data Augmentation

In [ ]:
#Add kerascv layers for random augmentation
rand_augment = keras_cv.layers.RandAugment(
    value_range=(0, 255),
    augmentations_per_image=3,
    magnitude=0.3,
    magnitude_stddev=0.2,
    rate=0.5,
)
rand_flip = keras_cv.layers.RandomFlip(mode='horizontal')
mix_up = keras_cv.layers.MixUp()

augmenter = keras_cv.layers.Augmenter(
    layers=[
        rand_augment,
        rand_flip,
        mix_up,
    ])

In [ ]:
def augment(images, y, augmenter):
    inputs = {"images": images, "labels": y}
    output = augmenter(inputs)
    return output["images"], output["labels"]

def process_validation(images, y):
    return images, y

def make_dataset(X, y, mode="train", batch_size=batch_size, augmenter=augmenter):
    ds = tf.data.Dataset.from_tensor_slices((X, y))
    if mode == "train":
        ds = ds.shuffle(batch_size * 4)
    ds = ds.batch(batch_size)
    if mode == "train":
        ds = ds.map(lambda x, y: augment(x, y, augmenter), num_parallel_calls=tf.data.AUTOTUNE)
    else:
        ds = ds.map(process_validation, num_parallel_calls=tf.data.AUTOTUNE)
        ds = ds.cache()
    ds = ds.prefetch(tf.data.AUTOTUNE)
    return ds

def visualize_dataset(dataset, row = 3, col = 3, title=None):
    for X, _ in dataset.take(1):
        fig, axs = plt.subplots(row, col, figsize=(5, 5))
        for i in range(row):
            for j in range(col):
              axs[i, j].imshow(X[i * row + j] / 255.0)
              axs[i, j].axis("off")
        if title is not None:
          fig.suptitle(title, fontsize=16)
        plt.show()


In [ ]:
train_ds = make_dataset(X_train * 255, y_train)
valid_ds = make_dataset(X_val * 255, y_val, mode="valid")

In [ ]:
visualize_dataset(train_ds, title='All augmentation')

## RandAugment

In [ ]:
rand_aug = keras_cv.layers.Augmenter(layers=[rand_augment])

train_ds_after_rand_aug = make_dataset(X_train * 255, y_train, augmenter=rand_aug)
visualize_dataset(train_ds_after_rand_aug, title='RandAugment')

## MixUp

In [ ]:
mix_up_aug = keras_cv.layers.Augmenter(layers=[mix_up])
train_ds_after_mix_up = make_dataset(X_train * 255, y_train, augmenter=mix_up_aug)
visualize_dataset(train_ds_after_mix_up, title='MixUp')

## RandFlip

In [ ]:
flip_aug = keras_cv.layers.Augmenter(layers=[rand_flip])

train_ds_after_flip = make_dataset(X_train * 255, y_train, augmenter=flip_aug)
visualize_dataset(train_ds_after_flip, title='RandomHorizontalFlip')

# Model

In [ ]:
#HyperSpace found thanks to the hyperparametere tuning
hs = {
    'gauss_noise': 0.1,
    'init': 'he_uniform',
    'units_1': 128,
    'units_2': 64,
    'dropout_1': 0.3,
    'learning_rate': 1e-4,
    'weight_decay': 5e-4,
    'n_layers': 2,
    'bnorm': True,
    'dropout': True,
    'alpha': alpha,
}

big_hs = {
    'gauss_noise': 0.1,
    'init': 'he_uniform',
    'units_1': 256,
    'units_2': 128,
    'dropout_1': 0.2,
    'learning_rate': 1e-4,
    'weight_decay': 5e-4,
    'n_layers': 2,
    'bnorm': True,
    'dropout': True,
    'alpha': alpha,
}

#Best model for the ensembling according to previous trainings 
baseline_models = {
    'convnext_xlarge': {
        'baseline_model': tfk.applications.ConvNeXtXLarge(
            weights='imagenet',
            include_top=False,
            input_shape=input_shape,
        ),
        'to_freeze': 228,
        'hs': big_hs,
        'weight': 0.4
    },
    'efficientnetv2-l': {
        'baseline_model': tfk.applications.EfficientNetV2L(
            weights='imagenet',
            include_top=False,
            input_shape=input_shape,
        ),
        'to_freeze': 663,
        'hs': hs,
        'weight': 0.2
    },
    'convnext_base': {
        'baseline_model': tfk.applications.ConvNeXtBase(
            weights='imagenet',
            include_top=False,
            input_shape=input_shape,
        ),
        'to_freeze': 220,
        'hs': big_hs,
        'weight': 0.4

    },
}

In [ ]:
def build_model(
    model_name,
    baseline_models,
    input_shape=input_shape,
    output_shape=output_shape,
):
    hs = baseline_models[model_name]['hs']
    
    preprocessing = tfk.Sequential([
        tfkl.GaussianNoise(hs['gauss_noise'], name=f'{model_name}_gauss_noise'),
    ], name=f'{model_name}_preprocessing')

    input_layer = tfkl.Input(shape=input_shape, name=f'{model_name}_input_layer')

    baseline_model = baseline_models[model_name]['baseline_model']

    baseline_model.trainable = False
    
    x = preprocessing(input_layer)

    x = baseline_model(x)

    x = tfkl.GlobalAveragePooling2D(name=f'{model_name}_gap')(x)

    x = tfkl.Dense(hs['units_1'], kernel_initializer=hs['init'], name=f'{model_name}_dense_1')(x)
    if (hs['bnorm']):
        x = tfkl.BatchNormalization(name=f'{model_name}_batch_norm')(x)
    x = tfkl.Activation(activation='relu', name=f'{model_name}_activation_1')(x)
    
    for i in range(1, hs['n_layers']):
        if (hs['dropout']):
            x = tfkl.Dropout(hs[f'dropout_{i}'], name=f'{model_name}_dropout_{i}')(x)

        x = tfkl.Dense(hs[f'units_{i+1}'], kernel_initializer=hs['init'], name=f'{model_name}_dense_{i+1}')(x)
        x = tfkl.Activation(activation='relu', name=f'{model_name}_activation_{i+1}')(x)
    
    

    
    output_layer = tfkl.Dense(output_shape, name=f'{model_name}_output_layer', activation='softmax')(x)

    # Create the model
    model = tfk.Model(inputs=input_layer, outputs=output_layer, name=model_name)
    
    #Using different loss to address the problem of class invariance
    loss = tfk.losses.CategoricalFocalCrossentropy(alpha=hs['alpha'])
    model.compile(loss=loss, optimizer=tfk.optimizers.AdamW(hs['learning_rate'], weight_decay=hs['weight_decay']), metrics=['accuracy'])
    return model

# Transfer learning
Perform transfer learnign for each models defined in baseline_models

In [ ]:
for model_name in baseline_models:        
    tl_model = build_model(model_name=model_name, baseline_models=baseline_models)
    tl_model.summary()
    
    train_ds = make_dataset(X_train * 255, y_train)
    valid_ds = make_dataset(X_val * 255, y_val, mode="valid")
    
    tl_history = tl_model.fit(
        train_ds,
        validation_data=valid_ds,
        epochs=epochs,
        batch_size=batch_size,
        verbose=1,
        callbacks=callbacks,
    ).history
    
    tl_model.save(f'{model_name}_model_tl')
    del tl_model

# Fine tuning
Fine tune previously pre-trained models

In [ ]:
for model_name in baseline_models.keys():
    ft_model = tf.keras.models.load_model(f'{model_name}_model_tl')
    hs = baseline_models[model_name]['hs']
    baseline_model = ft_model.get_layer(model_name)
    baseline_model.trainable = True
    for i, layer in enumerate(baseline_model.layers[:baseline_models[model_name]['to_freeze']+1]):
        layer.trainable=False
    # Create the model
    ft_model.compile(loss=tfk.losses.CategoricalFocalCrossentropy(alpha=hs['alpha']), optimizer=tfk.optimizers.AdamW(1e-5, weight_decay=5e-5), metrics=['accuracy'])
    ft_model.summary()
    # Fit the model
    train_ds = make_dataset(X_train * 255, y_train)
    valid_ds = make_dataset(X_val * 255, y_val, mode="valid")
    ft_history = ft_model.fit(
        train_ds,
        validation_data=valid_ds,
        epochs=epochs,
        batch_size=batch_size,
        verbose=1,
        callbacks=callbacks,
    ).history
    
    ft_model.save(f'{model_name}_model_ft')
    del ft_model

# Inference

In [ ]:
ft_models = [tf.keras.models.load_model(f'{model_name}_model_ft') for model_name in baseline_models]
preds = np.array([model.predict(X_test * 255, verbose=1) for model in ft_models])
weights = np.array([baseline_models[model_name]['weight'] for model_name in baseline_models])
preds.shape

In [ ]:
w_preds = np.array([pred * weights[i] for i, pred in enumerate(preds)])
w_preds.shape

In [ ]:
f_pred = np.argmax(np.sum(w_preds, axis=0), axis=-1)
f_pred.shape

In [ ]:
def compute_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred).round(4)
    precision = precision_score(y_true, y_pred, average='macro').round(4)
    recall = recall_score(y_true, y_pred, average='macro').round(4)
    f1 = f1_score(y_true, y_pred, average='macro').round(4)

    return {
      'accuracy': accuracy,
      'precision': precision,
      'recall' : recall,
      'f1' : f1
    }

def compute_cm(y_true, y_pred):
  # Compute the confusion matrix
  cm = confusion_matrix(y_true, y_pred)
  # Plot the confusion matrix
  plt.figure(figsize=(10, 8))
  sns.heatmap(cm.T, xticklabels=list(labels.values()), yticklabels=list(labels.values()), cmap='Blues', annot=True)
  plt.xlabel('True labels')
  plt.ylabel('Predicted labels')
  plt.show()

def make_inference(y_true, y_pred, display_cm=True):
    metrics = compute_metrics(y_true, y_pred)
    # Display the computed metrics
    print(metrics)
    if (display_cm):
        compute_cm(y_true, y_pred)

In [ ]:
make_inference(np.argmax(y_test, axis=-1), f_pred)

What we found out?
EfficientNet2v-l performs poorly with the current hyperparameter space, however, due to the lack of resources, we couldn't attempt to increase
its performance by means of KerasTuner Hyperband tuner (see other notebook)

# K Cross Validation
To determine the best epoch for each model in the notebook, that we will fine tune again using the entire dataset

In [ ]:
metadata = {}

for model_name in baseline_models.keys():
    # Define the number of folds for cross-validation
    num_folds = 5

    # Initialize lists to store training histories, scores, and best epochs
    histories = []
    scores = []
    best_epochs = []

    # Create a KFold cross-validation object
    kfold = KFold(n_splits=num_folds, shuffle=True, random_state=seed)
    # Loop through each fold
    for fold_idx, (train_idx, valid_idx) in enumerate(kfold.split(X_train_val, y_train_val)):
        print(f"Starting training model {model_name} on fold num: {fold_idx+1}")

        # Build a new dropout model for each fold
        k_model = tf.keras.models.load_model(f'{model_name}_model_tl')
        k_model.compile(
            loss=tfk.losses.CategoricalFocalCrossentropy(alpha=hs['alpha']), 
            optimizer=tfk.optimizers.AdamW(1e-5, weight_decay=5e-5),
            etrics=['accuracy']
        )

        train_ds = make_dataset(X_train_val[train_idx] * 255, y_train_val[train_idx])
        valid_ds = make_dataset(X_train_val[valid_idx] * 255, y_train_val[valid_idx], mode="valid")
        # Train the model on the training data for this fold
        history = k_model.fit(
            train_ds,
            validation_data=valid_ds,
            batch_size = batch_size,
            epochs = epochs,
            callbacks = callbacks,
            verbose = 1
          ).history
        
        
        # Evaluate the model on the validation data for this fold
        score = k_model.evaluate(valid_ds, verbose=0)
        print("score: {}".format(score))
        scores.append(score[1])

        # Calculate the best epoch for early stopping
        best_epoch = len(history['loss']) - es_patience
        best_epochs.append(best_epoch)

        # Store the training history for this fold
        histories.append(history)
        
    metadata[model_name] = {
        'histories': histories,
        'scores': scores,
        'best_epochs': best_epochs,
    }

# Plot histories

In [ ]:
avg_epochs = {}
for model_name, info in metadata.items():
    # Define a list of colors for plotting
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd']
    scores = info['scores']
    histories = info['histories']
    best_epochs = info['best_epochs']
    
    # Print mean and standard deviation of Accuracy scores
    print(f"Accuracy -> (mean: {np.mean(scores).round(4)}, std: {np.std(scores).round(4)})")

    # Create a figure for Accuracy visualization
    plt.figure(figsize=(15,6))
    # Plot val accuracy for each fold
    plt.title(f'{model_name} Validation Accuracy')
    for fold_idx in range(num_folds):
        print(histories[fold_idx]['val_accuracy'][:-es_patience])
        plt.plot(histories[fold_idx]['val_accuracy'][:-es_patience], color=colors[fold_idx], label=f'Fold N°{fold_idx+1}')
        plt.legend(loc='upper right')
        plt.grid(alpha=.3)

    # Show the plot
    plt.show()
    
    # Calculate the average best epoch
    avg_epoch = int(np.mean(best_epochs))
    print(f"{model_name} best average epoch: {avg_epoch}")
    avg_epochs[model_name] = avg_epoch


# Final training
Training with the best average

In [ ]:
#for each model training with the best average epoch
for model_name in baseline_models:
    print(f"{model_name} training")
    final_model = tf.keras.models.load_model(f"/kaggle/input/models-tl/{model_name}_model_tl")
    
    final_model.compile(
        loss=tfk.losses.CategoricalFocalCrossentropy(alpha=hs['alpha']), 
        optimizer=tfk.optimizers.AdamW(1e-5, weight_decay=5e-5),
        etrics=['accuracy']
    )
    final_model.summary()
    
    ds = make_dataset(X * 255, y)

    history = final_model.fit(
        ds,
        epochs = avg_epochs[model_name],
        callbacks = callbacks,
        verbose = 1
      ).history
    
    final_model.save(f'{model_name}_final_model')
    